<a href="https://colab.research.google.com/github/afshalliaquat/Sentimental_Analysis_NaiveBayes.ipynb/blob/main/Sentimental_Analysis_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('train.txt',sep = ';',header=None,names= ['text','emotions'])

In [5]:
unique_emotions = df['emotions'].unique()
emotion_no = {}
i=0

for emo in unique_emotions:
  emotion_no[emo]=i
  i+=1

df['emotions'] = df['emotions'].map(emotion_no)

In [6]:
df['text'] = df['text'].apply(lambda x:x.lower())

In [7]:
import string

In [8]:
def remove_punc(text):
  return text.translate(str.maketrans('','',string.punctuation))

In [9]:
df['text'] = df['text'].apply(remove_punc)

In [10]:
def remove_no(text):
  string = ''
  for i in text:
    if not i.isdigit():
      string += i
  return string


In [11]:
df['text'] = df['text'].apply(remove_no)

In [12]:
def remove_emoji(text):
  string = ''
  for i in text:
    if i.isascii():
      string+=i
  return string

In [13]:
df['text'] = df['text'].apply(remove_emoji)

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [15]:
nltk.download('stopwords')
nltk.download('punkt')       # for word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
stop_words = set(stopwords.words('english'))

In [17]:
def remove(txt):
  words = txt.split()
  cleaned = []
  for i in words:
    if not i in stop_words:
      cleaned.append(i)
  return ' '.join(cleaned)

In [18]:
df['text'] = df['text'].apply(remove)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X = df['text']
y= df['emotions']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer  #Bag of words (only 0 and 1s)

In [23]:
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

model_LR = LogisticRegression(max_iter=1000)
model_LR.fit(X_train_vector, y_train)
y_pred_LR = model_LR.predict(X_test_vector)
accuracy_score(y_test,y_pred_LR)

0.8628125

In [27]:
Parameter_LR = {
    'C': [0.01, 0.1, 1, 10, 100],     # C is for regularization strength
    'penalty': ['l2'],                # Only l2 works with most solvers
}

In [28]:
LR_para = RandomizedSearchCV(estimator=model_LR,cv=5,n_iter=5,param_distributions=Parameter_LR)
LR_para.fit(X_train_vector,y_train)
new_model_LR =LR_para.best_estimator_
y_pre = new_model_LR.predict(X_test_vector)
accuracy_score(y_test,y_pre)

0.88125

In [29]:
from sklearn.linear_model import LogisticRegression

model_LR_bow = LogisticRegression(max_iter=1000)
model_LR_bow.fit(X_train_bow, y_train)
y_pred_LR_bow = model_LR_bow.predict(X_test_bow)
accuracy_score(y_test,y_pred_LR_bow)

0.8896875

In [30]:
LR_para_bow = RandomizedSearchCV(estimator=model_LR_bow,cv=5,n_iter=5,param_distributions=Parameter_LR)
LR_para_bow.fit(X_train_bow,y_train)
new_model_LR_bow =LR_para_bow.best_estimator_
y_pre_bow = new_model_LR_bow.predict(X_test_bow)
accuracy_score(y_test,y_pre_bow)

0.8896875

In [32]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_LR_bow))
print(classification_report(y_test, y_pred_LR))
print(classification_report(y_test, y_pre_bow))
print(classification_report(y_test, y_pre))


              precision    recall  f1-score   support

           0       0.93      0.93      0.93       946
           1       0.89      0.86      0.87       427
           2       0.85      0.76      0.80       296
           3       0.86      0.73      0.79       113
           4       0.86      0.84      0.85       397
           5       0.88      0.94      0.91      1021

    accuracy                           0.89      3200
   macro avg       0.88      0.84      0.86      3200
weighted avg       0.89      0.89      0.89      3200

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       946
           1       0.90      0.81      0.86       427
           2       0.90      0.61      0.73       296
           3       0.88      0.47      0.61       113
           4       0.86      0.76      0.81       397
           5       0.81      0.96      0.88      1021

    accuracy                           0.86      3200
   macro avg       0.88

In [ ]:
import joblib

joblib.dump(models['model_LR'],'LR_sentamental_analysis.pkl')
joblib.dump(scaler,'scaler.pkl')
joblib.dump(x.columns.to_list(),'columns.pkl')